In [122]:
from atproto import Client
import requests
import pandas as pd

from tqdm import tqdm

In [123]:
with open('data/bsky_credentials.txt', 'r') as file:
    username = file.readline().strip()
    password = file.readline().strip()

In [124]:
# client = Client()
# client.login(username, password)
# client.send_post(text='Hello World!')

In [125]:
def get_profile(username):
    response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.actor.getProfile?actor=" + username)
    data = response.json()
    return data

Average number of followers

Average number of followings

Number of reposts

Number of tweets

Same but in 1 hour

Average Time Diffrence : time between a tweet and its repost

Time of first tweet

Time of last tweet

Number of favourites

Number of uniuqe users in 10 hours

In [126]:
profile = get_profile("mjfree.bsky.social")

In [127]:
profile['followersCount']

270121

In [128]:
profile['followsCount']

1963

In [147]:
uri = "at://did:plc:gkgmduxh722ocstroyi75gbg/app.bsky.feed.post/3le56mzpymk2f"

In [148]:
def get_post(uri):
    response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.feed.getPosts?uris=" + uri)
    data = response.json()
    return data

In [149]:
post = get_post(uri)
post

{'posts': [{'uri': 'at://did:plc:gkgmduxh722ocstroyi75gbg/app.bsky.feed.post/3le56mzpymk2f',
   'cid': 'bafyreiazqnepgmtpwqq7e2m6jsjalkfprr2qexzfqacboa7slixgq3fesi',
   'author': {'did': 'did:plc:gkgmduxh722ocstroyi75gbg',
    'handle': 'mjfree.bsky.social',
    'displayName': 'Morgan J Freeman',
    'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:gkgmduxh722ocstroyi75gbg/bafkreifwct23l4uc5htaf3dclknrfg3mo4l3lqno4wl7ce5c5byzgl5v4m@jpeg',
    'labels': [],
    'createdAt': '2024-11-09T03:00:34.077Z'},
   'record': {'$type': 'app.bsky.feed.post',
    'createdAt': '2024-12-25T14:36:00.018Z',
    'langs': ['en'],
    'text': 'Merry Fucking Christmas 🎄'},
   'replyCount': 43,
   'repostCount': 38,
   'likeCount': 626,
   'quoteCount': 4,
   'indexedAt': '2024-12-25T14:36:00.248Z',
   'labels': []}]}

In [150]:
def get_reposts(uri, limit=100):
    response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.feed.getRepostedBy?uri=" + uri + "&limit=" + str(limit))
    data = response.json()
    return data

In [151]:
reposts = get_reposts(uri, limit=100)
reposts

{'repostedBy': [{'did': 'did:plc:zxn4wqkjmzbf5b3utmjebe2u',
   'handle': 'randomfurryfan.bsky.social',
   'displayName': 'Lesbian Furry Mayhem',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zxn4wqkjmzbf5b3utmjebe2u/bafkreidkvms2vms6uakyvfwop4kebn2awe26ttmkvpfm2gbiephutxfoa4@jpeg',
   'labels': [],
   'createdAt': '2024-11-21T22:59:05.135Z',
   'description': 'President Of Furkistan\n#1 Amy Rose Fan\nTikTok: www.tiktok.com/@furrystuffbecauseiwantto',
   'indexedAt': '2024-12-23T18:58:25.743Z'},
  {'did': 'did:plc:zn7lhrqjrbao6flovgbxwbup',
   'handle': 'amsterdamred-5280.bsky.social',
   'displayName': 'amsterdamred**NOT LOOKING FOR LOVE or Bitcoin or $$investment**',
   'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zn7lhrqjrbao6flovgbxwbup/bafkreiew7knjasi34e5jwzlarpkqr7jysfcxszlqexljwv6bjsvsnw25da@jpeg',
   'labels': [],
   'createdAt': '2024-11-13T19:42:29.573Z',
   'description': 'I live a Mile HIGH 5280 here in Denver, Colorado, with my two dogs. Big Red 1

In [152]:
def get_feed(username, limit=100):
    response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.feed.getAuthorFeed?actor=" + username + "&limit=" + str(limit))
    data = response.json()
    return data


In [154]:
feed = get_feed("randomfurryfan.bsky.social", limit=100)
feed

{'feed': [{'post': {'uri': 'at://did:plc:zxn4wqkjmzbf5b3utmjebe2u/app.bsky.feed.post/3le5rs5osg226',
    'cid': 'bafyreictdvsdvwxmbdg64ovagpggfybae55r7r2bhqx6ooudt25x466jde',
    'author': {'did': 'did:plc:zxn4wqkjmzbf5b3utmjebe2u',
     'handle': 'randomfurryfan.bsky.social',
     'displayName': 'Lesbian Furry Mayhem',
     'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:zxn4wqkjmzbf5b3utmjebe2u/bafkreidkvms2vms6uakyvfwop4kebn2awe26ttmkvpfm2gbiephutxfoa4@jpeg',
     'labels': [],
     'createdAt': '2024-11-21T22:59:05.135Z'},
    'record': {'$type': 'app.bsky.feed.post',
     'createdAt': '2024-12-25T20:18:53.039Z',
     'embed': {'$type': 'app.bsky.embed.images',
      'images': [{'alt': '',
        'aspectRatio': {'height': 90, 'width': 728},
        'image': {'$type': 'blob',
         'ref': {'$link': 'bafkreiceik2fkuzlgup5qn3bxyzebxtwqztye4fga2jsi7g3xarewdwazu'},
         'mimeType': 'image/jpeg',
         'size': 149628}}]},
     'langs': ['en'],
     'text': 'She got ba

In [38]:
def search_posts(query, limit=100):
    response = requests.get("https://public.api.bsky.app/xrpc/app.bsky.feed.searchPosts?q=" + query + "&limit=" + str(limit))
    data = response.json()
    return data

In [43]:
query = "macron trump"

In [ ]:
posts = search_posts(query)
posts

{'posts': [{'uri': 'at://did:plc:mr2ugr2pnno4g4i7fli2hdyl/app.bsky.feed.post/3le5hgrkza22k',
   'cid': 'bafyreias7emfo3a2rper6qh4lbxlivrlhawwaqxyekbuohhbgd3pcb4gpe',
   'author': {'did': 'did:plc:mr2ugr2pnno4g4i7fli2hdyl',
    'handle': 'woody68.bsky.social',
    'displayName': 'Woody68',
    'avatar': 'https://cdn.bsky.app/img/avatar/plain/did:plc:mr2ugr2pnno4g4i7fli2hdyl/bafkreid4xmg6piigx3yiprnkafepa7aiwxlgh6v5l3b6nbskwcggotjqy4@jpeg',
    'associated': {'chat': {'allowIncoming': 'following'}},
    'labels': [],
    'createdAt': '2023-12-01T14:00:56.250Z'},
   'record': {'$type': 'app.bsky.feed.post',
    'createdAt': '2024-12-25T17:13:33.816Z',
    'langs': ['en'],
    'text': 'I urge Biden, Macron, Starmer, Støre, Fredriksen and more of the western democracies to give Ukraine possibility of drowning Russia with missiles, aircrafts and everything else they need, until Trump enters WH.'},
   'replyCount': 0,
   'repostCount': 0,
   'likeCount': 0,
   'quoteCount': 0,
   'indexedAt':

In [45]:
len(posts["posts"])

100

In [116]:
politifact_fake = pd.read_csv("FakeNewsNet/politifact_fake.csv")
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [117]:
politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [100]:
politifact_fake[politifact_fake['id'] == "politifact15156"]['title']

1    Court Orders Obama To Pay $400 Million In Rest...
Name: title, dtype: object

In [118]:
def create_dataset(news, limit=100):
    
    dataset = {"post_uri": [],
                "post_cid": [],
                "type": "post",
                "date": [],
                "news_id": [],
                "like_count": [],
                "repost_count": [],
                "user_name": [],
                "follower_count": [],
                "follows_count": [],
                }
    
    for query in tqdm(news["title"]):
        posts = search_posts(query, limit)
        for post in posts["posts"]:
            profile = get_profile(post["author"]["handle"])
            dataset["post_uri"].append(post["uri"])
            dataset["post_cid"].append(post["cid"])
            dataset["date"].append(post["record"]["createdAt"])
            dataset["news_id"].append(news[news["title"] == query]["id"].values[0])
            dataset["like_count"].append(post["likeCount"])
            dataset["repost_count"].append(post["repostCount"])
            dataset["user_name"].append(post["author"]["handle"])
            dataset["follower_count"].append(profile["followersCount"])
            dataset["follows_count"].append(profile["followsCount"])
            
    dataframe = pd.DataFrame(dataset, columns=["post_uri", "post_cid", "type", "date", "news_id", "like_count", "repost_count", "user_name", "follower_count", "follows_count"])
            
    return dataframe

In [132]:
dataset_politifact_fake = create_dataset(politifact_fake, 10)
dataset_politifact_fake.head()

100%|██████████| 432/432 [08:29<00:00,  1.18s/it]


,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:42bvga7of7n2h354nhhcajpm/app.bsky...,bafyreiayjg2uhymqs5pop6eokdkbrcbrvttxtopydqbbp...,post,2024-12-25T21:31:11.499Z,"5 politifact14404 Name: id, dtype: object",0,0,princessfluffypaws.bsky.social,23,15
1,at://did:plc:qejc5g3on7wkr5axbvnraaml/app.bsky...,bafyreic4wn675il3kbzidjyskiqneqmsr4okjgsrfyyza...,post,2024-12-25T21:31:08.216Z,"5 politifact14404 Name: id, dtype: object",0,0,karenbdc.bsky.social,176,278
2,at://did:plc:2et5dxb5wl7p2qo635an2woy/app.bsky...,bafyreifxgrp2eoltduvogyrer6kcsmxqm3wiy6g3h7iyq...,post,2024-12-25T21:30:48.501Z,"5 politifact14404 Name: id, dtype: object",0,0,happy213.bsky.social,3278,2845
3,at://did:plc:sm7jmhl6rwoke2pgqpcoflm4/app.bsky...,bafyreia37ak6ovx47bwy4ypxynblmmobqigut2kzhxbrz...,post,2024-12-25T21:30:10.815Z,"5 politifact14404 Name: id, dtype: object",1,0,voidows.bsky.social,84,0
4,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,post,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,allie-froglegs.bsky.social,14191,19623


In [133]:
dataset_politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        196 non-null    object
 1   post_cid        196 non-null    object
 2   type            196 non-null    object
 3   date            196 non-null    object
 4   news_id         196 non-null    object
 5   like_count      196 non-null    int64 
 6   repost_count    196 non-null    int64 
 7   user_name       196 non-null    object
 8   follower_count  196 non-null    int64 
 9   follows_count   196 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 15.4+ KB


In [111]:
politifact_real = pd.read_csv("FakeNewsNet/politifact_real.csv")

In [ ]:
politifact_real.info()

In [112]:
dataset_politifact_real = create_dataset(politifact_real, 10)

  0%|          | 0/624 [00:00<?, ?it/s]

query National Federation of Independent Business


  1%|          | 6/624 [00:07<11:11,  1.09s/it]

query 25th Amendment


  1%|          | 7/624 [00:10<19:04,  1.86s/it]

...

query Change We Can Believe In


100%|█████████▉| 623/624 [18:43<00:01,  1.34s/it]

query Interest Group Ratings


100%|██████████| 624/624 [18:45<00:00,  1.80s/it]

Number of different news 159


In [113]:
dataset_politifact_real.head()

,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:ngygrbhyzeyigjlxnli3r2xw/app.bsky...,bafyreiaim3w4bcbajxupci7vfnw67ozr4cmy72ttheimf...,post,2024-12-22T14:05:12.057Z,"0 politifact14984 Name: id, dtype: object",1,0,themoneycruncher.bsky.social,1049,102
1,at://did:plc:il7345tnmrhnzaufvgk5qfcx/app.bsky...,bafyreif2fveaw7subm7yfe6avtzkucglpvibgp2qols7o...,post,2024-12-12T21:35:44.118Z,"0 politifact14984 Name: id, dtype: object",1,0,srubenfeld.bsky.social,845,321
2,at://did:plc:sk44rhbmaca3ltsa4c6v6da2/app.bsky...,bafyreigvqyv6ih42pbugjdg3enkbidvu7etu3kstqxtwb...,post,2024-12-10T14:34:58Z,"0 politifact14984 Name: id, dtype: object",1,0,worldnewsguru.bsky.social,174,0
3,at://did:plc:vyytbbzsqq4ihfgpdlgyjwc3/app.bsky...,bafyreieh4m2ej3nw2s3amso4hxlw2kk6fqoblryw5ooh2...,post,2024-12-10T14:06:59Z,"0 politifact14984 Name: id, dtype: object",0,0,newsw0rld.bsky.social,34,19
4,at://did:plc:7bgttf5dscwdbpn5r67ig7zb/app.bsky...,bafyreiefdcvelgytdvv2fea24waarueb7whoeyma7756u...,post,2024-12-10T12:53:50.145Z,"0 politifact14984 Name: id, dtype: object",1,0,mjm314.bsky.social,781,1317


In [114]:
dataset_politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        1228 non-null   object
 1   post_cid        1228 non-null   object
 2   type            1228 non-null   object
 3   date            1228 non-null   object
 4   news_id         1228 non-null   object
 5   like_count      1228 non-null   int64 
 6   repost_count    1228 non-null   int64 
 7   user_name       1228 non-null   object
 8   follower_count  1228 non-null   int64 
 9   follows_count   1228 non-null   int64 
dtypes: int64(4), object(6)
memory usage: 96.1+ KB


In [169]:
def add_reposts(posts_dataset, limit=100):
    
    dataset = {"post_uri": [],
                "post_cid": [],
                "type": "repost",
                "retweet_date": [],
                "news_id": [],
                "like_count": [],
                "repost_count": [],
                "user_name": [],
                "follower_count": [],
                "follows_count": [],
                }
    
    
    for query in tqdm(posts_dataset["post_uri"]):
        reposts = get_reposts(query, limit)
        for user in reposts["repostedBy"]:
            profile = get_profile(user["handle"])
            
            dataset["post_uri"].append(posts_dataset[posts_dataset["post_uri"] == query]["post_uri"].values[0])
            dataset["post_cid"].append(posts_dataset[posts_dataset["post_uri"] == query]["post_cid"].values[0])
            dataset["retweet_date"].append(posts_dataset[posts_dataset["post_uri"] == query]["date"].values[0])
            dataset["news_id"].append(posts_dataset[posts_dataset["post_uri"] == query]["news_id"].values[0])
            dataset["like_count"].append(posts_dataset[posts_dataset["post_uri"] == query]["like_count"].values[0])
            dataset["repost_count"].append(posts_dataset[posts_dataset["post_uri"] == query]["repost_count"].values[0])
            dataset["user_name"].append(user["handle"])
            dataset["follower_count"].append(profile["followersCount"])
            dataset["follows_count"].append(profile["followsCount"])
            
    dataframe = pd.DataFrame(dataset, columns=["post_uri", "post_cid", "type", "retweet_date", "news_id", "like_count", "repost_count", "user_name", "follower_count", "follows_count"])
            
    return dataframe

In [166]:
dataset_politifact_fake["post_uri"].loc[0]

'at://did:plc:42bvga7of7n2h354nhhcajpm/app.bsky.feed.post/3le5vth5zsk2m'

In [168]:
dataset_politifact_fake[dataset_politifact_fake["post_uri"] == "at://did:plc:42bvga7of7n2h354nhhcajpm/app.bsky.feed.post/3le5vth5zsk2m"]["like_count"].values[0]

0

In [170]:
politifact_fake_reposts = add_reposts(dataset_politifact_fake, 10)

100%|██████████| 196/196 [00:59<00:00,  3.29it/s]


In [171]:
politifact_fake_reposts.head()

,post_uri,post_cid,type,retweet_date,news_id,like_count,repost_count,user_name,follower_count,follows_count
0,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,repost,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,kyotiwiles.bsky.social,3713,4329
1,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,repost,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,nb01.bsky.social,1541,1681
2,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,repost,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,dianademarest.bsky.social,5875,5432
3,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,repost,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,genirae.bsky.social,5807,9135
4,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,repost,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,grahamwakeham.bsky.social,1055,2224


In [172]:
politifact_fake_reposts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        125 non-null    object
 1   post_cid        125 non-null    object
 2   type            125 non-null    object
 3   retweet_date    125 non-null    object
 4   news_id         125 non-null    object
 5   like_count      125 non-null    int64 
 6   repost_count    125 non-null    int64 
 7   user_name       125 non-null    object
 8   follower_count  125 non-null    int64 
 9   follows_count   125 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 9.9+ KB


In [173]:
politifact_fake_posts = pd.concat([dataset_politifact_fake, politifact_fake_reposts])

In [174]:
politifact_fake_posts.head()

,post_uri,post_cid,type,date,news_id,like_count,repost_count,user_name,follower_count,follows_count,retweet_date
0,at://did:plc:42bvga7of7n2h354nhhcajpm/app.bsky...,bafyreiayjg2uhymqs5pop6eokdkbrcbrvttxtopydqbbp...,post,2024-12-25T21:31:11.499Z,"5 politifact14404 Name: id, dtype: object",0,0,princessfluffypaws.bsky.social,23,15,NaN
1,at://did:plc:qejc5g3on7wkr5axbvnraaml/app.bsky...,bafyreic4wn675il3kbzidjyskiqneqmsr4okjgsrfyyza...,post,2024-12-25T21:31:08.216Z,"5 politifact14404 Name: id, dtype: object",0,0,karenbdc.bsky.social,176,278,NaN
2,at://did:plc:2et5dxb5wl7p2qo635an2woy/app.bsky...,bafyreifxgrp2eoltduvogyrer6kcsmxqm3wiy6g3h7iyq...,post,2024-12-25T21:30:48.501Z,"5 politifact14404 Name: id, dtype: object",0,0,happy213.bsky.social,3278,2845,NaN
3,at://did:plc:sm7jmhl6rwoke2pgqpcoflm4/app.bsky...,bafyreia37ak6ovx47bwy4ypxynblmmobqigut2kzhxbrz...,post,2024-12-25T21:30:10.815Z,"5 politifact14404 Name: id, dtype: object",1,0,voidows.bsky.social,84,0,NaN
4,at://did:plc:rcorabkatv6cnulby4gkkzsg/app.bsky...,bafyreiecdrapk7pofhow7swb5p6qqhtzeposslxfnicyf...,post,2024-12-25T21:28:18.466Z,"5 politifact14404 Name: id, dtype: object",5,2,allie-froglegs.bsky.social,14191,19623,NaN


In [175]:
politifact_fake_posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 321 entries, 0 to 124
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_uri        321 non-null    object
 1   post_cid        321 non-null    object
 2   type            321 non-null    object
 3   date            196 non-null    object
 4   news_id         321 non-null    object
 5   like_count      321 non-null    int64 
 6   repost_count    321 non-null    int64 
 7   user_name       321 non-null    object
 8   follower_count  321 non-null    int64 
 9   follows_count   321 non-null    int64 
 10  retweet_date    125 non-null    object
dtypes: int64(4), object(7)
memory usage: 30.1+ KB


In [177]:
politifact_fake_posts.groupby("news_id")["type"].value_counts()

TypeError: unhashable type: 'Series'

In [183]:
politifact_fake_posts.groupby("news_id")["follower_count"].mean()

TypeError: unhashable type: 'Series'

In [192]:
average = politifact_fake_posts.groupby(["news_id"])["follower_count"].
average

AttributeError: 'SeriesGroupBy' object has no attribute 'values'

In [ ]:
def get_features(dataframe):
    average_followers = dataframe.groupby("news_id")["follower_count"].mean()
    average_follows = dataframe.groupby("news_id")["follows_count"].mean()
    retweet_percentage = dataframe.groupby("news_id")["type"].value_counts().unstack()["repost"]

In [ ]:
politifact_real_reposts = add_reposts(dataset_politifact_real, 10)

In [ ]:
politifact_real_reposts.head()

In [ ]:
politifact_real_reposts.info()

In [ ]:
politifact_real_posts = pd.concat([dataset_politifact_real, politifact_real_reposts])